In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Install the spotipy library
%pip install spotipy==2.23.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.5/261.5 kB 6.9 MB/s eta 0:00:00


In [ ]:
# Authentication
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import time
import matplotlib.pyplot as plt
import seaborn as sns

In [34]:
# #setting up cred
# import spotipy
# from spotipy.oauth2 import SpotifyOAuth

# CLIENT_ID = "24034e4f22914755a3091fac35295b36"
# CLIENT_SECRET = "8817607c2bff46b4a0867d9f18d0af3d"
# REDIRECT_URI = "http://localhost:8888/callback"  # Keep this default

# sp = spotipy.Spotify(auth_manager=SpotifyOAuth(
#     client_id=CLIENT_ID,
#     client_secret=CLIENT_SECRET,
#     redirect_uri=REDIRECT_URI,
#     scope="user-library-read playlist-read-private audio-features"
# ))


In [35]:
# # Authenticate
# client_id = "24034e4f22914755a3091fac35295b36"
# client_secret = "8817607c2bff46b4a0867d9f18d0af3d"
# sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=client_id, client_secret=client_secret))

In [40]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
from tqdm import tqdm
import time

# Set up authentication
client_credentials_manager = SpotifyClientCredentials(
    client_id="24034e4f22914755a3091fac35295b36",
    client_secret="8817607c2bff46b4a0867d9f18d0af3d"
)

# Create Spotify client
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

# Test the connection by searching for a track
results = sp.search(q='year:2023', type='track', limit=1)
print("Successfully connected to Spotify API!")
print("\
Sample Track:", results['tracks']['items'][0]['name'])

Successfully connected to Spotify API!
Sample Track: No Pole


In [41]:
def extract_large_tracks_dataset(total_tracks=10000, batch_size=50):
    """Extract a large number of tracks with progress tracking"""
    all_tracks = []
    iterations = total_tracks // batch_size

    # Create years range for more diverse data
    years = list(range(2010, 2024))

    with tqdm(total=total_tracks, desc="Fetching tracks") as pbar:
        for year in years:
            offset = 0
            while len(all_tracks) < total_tracks and offset < 1000:  # Spotify limits to 1000 results per query
                try:
                    results = sp.search(
                        q=f'year:{year}',
                        type='track',
                        limit=batch_size,
                        offset=offset
                    )

                    if not results['tracks']['items']:
                        break

                    for track in results['tracks']['items']:
                        if len(all_tracks) >= total_tracks:
                            break

                        track_info = {
                            'id': track['id'],
                            'name': track['name'],
                            'artist': ','.join([artist['name'] for artist in track['artists']]),
                            'artist_id': ','.join([artist['id'] for artist in track['artists']]),
                            'album': track['album']['name'],
                            'popularity': track['popularity'],
                            'release_date': track['album']['release_date'],
                            'duration_ms': track['duration_ms'],
                            'explicit': track['explicit'],
                            'preview_url': track['preview_url'],
                            'external_url': track['external_urls']['spotify']
                        }
                        all_tracks.append(track_info)
                        pbar.update(1)

                    offset += batch_size
                    time.sleep(0.1)  # Rate limiting

                except Exception as e:
                    print(f"Error at offset {offset} for year {year}: {str(e)}")
                    time.sleep(1)  # Wait longer if we hit an error
                    continue

            if len(all_tracks) >= total_tracks:
                break

    df = pd.DataFrame(all_tracks)
    return df

# Extract tracks
print("Starting extraction of 10,000 tracks...")
tracks_df = extract_large_tracks_dataset(10000)

# Basic statistics
print("\
Dataset Overview:")
print("Total tracks collected:", len(tracks_df))
print("\
Sample of collected data:")
print(tracks_df.head())

# Save to CSV
tracks_df.to_csv('spotify_10k_tracks.csv', index=False)
print("\
Data saved to spotify_10k_tracks.csv")

Starting extraction of 10,000 tracks...


Fetching tracks: 100%|██████████| 10000/10000 [02:00<00:00, 82.93it/s]


Dataset Overview:
Total tracks collected: 10000
Sample of collected data:
                       id                  name                        artist  \
0  7BqBn9nzAq8spo5e7cZ0dJ  Just the Way You Are                    Bruno Mars   
1  0JXXNGljqupsJaZsgSbMZV            Sure Thing                        Miguel   
2  7FAFkQQZFeNwOFzTrSDFIh             The Spins  Mac Miller,Empire Of The Sun   
3  42qNWdLKCI41S4uzfamhFM                 Duvet                           bôa   
4  2tJulUYLDKOg9XrtVkMgcJ               Grenade                    Bruno Mars   

                                       artist_id                 album  \
0                         0du5cEVh5yTK9QJze8zA0C  Doo-Wops & Hooligans   
1                         360IAlyVv4PCEVjgyMZrxK     All I Want Is You   
2  4LLpKhyESsyAXpc4laK94U,67hb7towEyKvt5Z8Bx306c     K.I.D.S. (Deluxe)   
3                         5jTtGLk1mGFMY5lQOvJYUj              Twilight   
4                         0du5cEVh5yTK9QJze8zA0C  Doo-Wops & Hooligan